# helper funcs from https://www.kaggle.com/its7171/metrics-evaluation-script and https://www.kaggle.com/hocop1/centernet-baseline with some change and some constants

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import sqrt, acos, pi, sin, cos
from scipy.spatial.transform import Rotation as R
from sklearn.metrics import average_precision_score
from multiprocessing import Pool
import cv2
import matplotlib.pyplot as plt
from math import sin, cos, sqrt

submission_path_name1 = '../input/center-efficientnet-submission/predictions.csv'
submission_path_name2 = '../input/masked-centernet-baseline-submisstion/predictions.csv'
test_images_dir = '../input/pku-autonomous-driving/test_images/'
TD_thr = 0.05
RD_thr = 10

# the real threshold is (5400/distance)*ID_thr_factor, where distance is the distance from the camera to a current object.
ID_thr_factor = 0.3
use_ID_thr = True

#merging_plan = 'non-max-suppress'
merging_plan = 'weighted-add'

# From camera.zip
camera_matrix = np.array([[2304.5479, 0,  1686.2379],
                          [0, 2305.8757, 1354.9849],
                          [0, 0, 1]], dtype=np.float32)

def expand_df(df, PredictionStringCols):
    df = df.dropna().copy()
    df['NumCars'] = [int((x.count(' ')+1)/7) for x in df['PredictionString']]

    image_id_expanded = [item for item, count in zip(df['ImageId'], df['NumCars']) for i in range(count)]
    prediction_strings_expanded = df['PredictionString'].str.split(' ',expand = True).values.reshape(-1,7).astype(float)
    prediction_strings_expanded = prediction_strings_expanded[~np.isnan(prediction_strings_expanded).all(axis=1)]
    df = pd.DataFrame(
        {
            'ImageId': image_id_expanded,
            PredictionStringCols[0]:prediction_strings_expanded[:,0],
            PredictionStringCols[1]:prediction_strings_expanded[:,1],
            PredictionStringCols[2]:prediction_strings_expanded[:,2],
            PredictionStringCols[3]:prediction_strings_expanded[:,3],
            PredictionStringCols[4]:prediction_strings_expanded[:,4],
            PredictionStringCols[5]:prediction_strings_expanded[:,5],
            PredictionStringCols[6]:prediction_strings_expanded[:,6]
        })
    return df

def str2coords(s, names=['yaw', 'pitch', 'roll', 'x', 'y', 'z', 'confidence']):
    coords = []
    for l in np.array(s.split()).reshape([-1, 7]):
        coords.append(dict(zip(names, l.astype('float'))))
    return coords

def TranslationDistance(p,g, abs_dist = False):
    dx = p['x'] - g['x']
    dy = p['y'] - g['y']
    dz = p['z'] - g['z']
    diff0 = (g['x']**2 + g['y']**2 + g['z']**2)**0.5
    diff1 = (dx**2 + dy**2 + dz**2)**0.5
    if abs_dist:
        diff = diff1
    else:
        diff = diff1/diff0
    return diff

def RotationDistance(p, g):
    true=[ g['pitch'] ,g['yaw'] ,g['roll'] ]
    pred=[ p['pitch'] ,p['yaw'] ,p['roll'] ]
    q1 = R.from_euler('xyz', true)
    q2 = R.from_euler('xyz', pred)
    diff = R.inv(q2) * q1
    W = np.clip(diff.as_quat()[-1], -1., 1.)
    
    # in the official metrics code:
    # https://www.kaggle.com/c/pku-autonomous-driving/overview/evaluation
    #   return Object3D.RadianToDegree( Math.Acos(diff.W) )
    # this code treat θ and θ+2π differntly.
    # So this should be fixed as follows.
    W = (acos(W)*360)/pi
    if W > 180:
        W = 360 - W
    return W

def coords2str(coords, names=['yaw', 'pitch', 'roll', 'x', 'y', 'z', 'confidence']):
    s = []
    for c in coords:
        for n in names:
            s.append(str(c.get(n, 0)))
    return ' '.join(s)

def imread(path, fast_mode=False):
    img = cv2.imread(path)
    if not fast_mode and img is not None and len(img.shape) == 3:
        img = np.array(img[:, :, ::-1])
    return img

def get_img_coords(inp):
    '''
    Input is a PredictionString (e.g. from train dataframe) or a coordinate list
    Output is two arrays:
        xs: x coordinates in the image (row)
        ys: y coordinates in the image (column)
    '''
    if isinstance(inp, str):
        coords = str2coords(inp)
    else:
        coords = inp
    xs = [c['x'] for c in coords]
    ys = [c['y'] for c in coords]
    zs = [c['z'] for c in coords]
    P = np.array(list(zip(xs, ys, zs))).T
    img_p = np.dot(camera_matrix, P).T
    img_p[:, 0] /= img_p[:, 2]
    img_p[:, 1] /= img_p[:, 2]
    img_xs = img_p[:, 0]
    img_ys = img_p[:, 1]
    img_zs = img_p[:, 2] # z = Distance from the camera
    return img_xs, img_ys

# convert euler angle to rotation matrix
def euler_to_Rot(yaw, pitch, roll):
    Y = np.array([[cos(yaw), 0, sin(yaw)],
                  [0, 1, 0],
                  [-sin(yaw), 0, cos(yaw)]])
    P = np.array([[1, 0, 0],
                  [0, cos(pitch), -sin(pitch)],
                  [0, sin(pitch), cos(pitch)]])
    R = np.array([[cos(roll), -sin(roll), 0],
                  [sin(roll), cos(roll), 0],
                  [0, 0, 1]])
    return np.dot(Y, np.dot(P, R))

def draw_line(image, points):
    color = (255, 0, 0)
    cv2.line(image, tuple(points[0][:2]), tuple(points[3][:2]), color, 16)
    cv2.line(image, tuple(points[0][:2]), tuple(points[1][:2]), color, 16)
    cv2.line(image, tuple(points[1][:2]), tuple(points[2][:2]), color, 16)
    cv2.line(image, tuple(points[2][:2]), tuple(points[3][:2]), color, 16)
    return image


def draw_points(image, points):
    for (p_x, p_y, p_z) in points:
        cv2.circle(image, (p_x, p_y), int(1000 / p_z), (0, 255, 0), -1)
#         if p_x > image.shape[1] or p_y > image.shape[0]:
#             print('Point', p_x, p_y, 'is out of image with shape', image.shape)
    return image

def visualize(img, coords):
    # You will also need functions from the previous cells
    x_l = 1.02
    y_l = 0.80
    z_l = 2.31
    
    img = img.copy()
    for point in coords:
        # Get values
        x, y, z = point['x'], point['y'], point['z']
        yaw, pitch, roll = -point['pitch'], -point['yaw'], -point['roll']
        # Math
        Rt = np.eye(4)
        t = np.array([x, y, z])
        Rt[:3, 3] = t
        Rt[:3, :3] = euler_to_Rot(yaw, pitch, roll).T
        Rt = Rt[:3, :]
        P = np.array([[x_l, -y_l, -z_l, 1],
                      [x_l, -y_l, z_l, 1],
                      [-x_l, -y_l, z_l, 1],
                      [-x_l, -y_l, -z_l, 1],
                      [0, 0, 0, 1]]).T
        img_cor_points = np.dot(camera_matrix, np.dot(Rt, P))
        img_cor_points = img_cor_points.T
        img_cor_points[:, 0] /= img_cor_points[:, 2]
        img_cor_points[:, 1] /= img_cor_points[:, 2]
        img_cor_points = img_cor_points.astype(int)
        # Drawing
        img = draw_line(img, img_cor_points)
        img = draw_points(img, img_cor_points[-1:])
    
    return img

In [ ]:
if not use_ID_thr:
    def is_close_enough(a, b, TD_thr=TD_thr, RD_thr=RD_thr):
        # judge if a and b are close enough.

        RD = RotationDistance(a, b)
        if RD > RD_thr:
            return False, None, None
        TD = TranslationDistance(a, b)
        if TD > TD_thr:
            return False, None, None
        return True, TD, RD
else:
    # use image coordinates
    def is_close_enough(a, b, thr_factor=ID_thr_factor):
        # judge if a and b are close enough.

        RD = RotationDistance(a, b)
        if RD > RD_thr:
            return False, None, None

        img_x_a, img_y_a = get_img_coords([a])
        img_x_b, img_y_b = get_img_coords([b])
        distance = sqrt((img_x_a-img_x_b)*(img_x_a-img_x_b)+(img_y_a-img_y_b)*(img_y_a-img_y_b))
        #print(distance)

        if distance > (5400.0/a['z'])*thr_factor:
            return False, None, None

        return True, distance, None

# merging

In [ ]:
submission1 = pd.read_csv(submission_path_name1)
submission2 = pd.read_csv(submission_path_name2)

# to avoid length difference
if len(submission1) > len(submission2):
    main_submission = submission1
    auxiliary_submission = submission2
else:
    main_submission = submission2
    auxiliary_submission = submission1

new_submission = main_submission.copy()    
for i in range(len(auxiliary_submission)):
    AS_predictions = auxiliary_submission.loc[i, 'PredictionString']
    #AS_predictions = auxiliary_submission.loc[auxiliary_submission['ImageId']=='ID_001c3cc69', 'PredictionString']
    if isinstance(AS_predictions, float):
        A_coords = []
    else:
        A_coords = str2coords(AS_predictions)
        
    MS_predictions = main_submission.loc[main_submission['ImageId']==auxiliary_submission.loc[i,'ImageId'], 'PredictionString'].values[0]
    #MS_predictions = main_submission.loc[main_submission['ImageId']=='ID_001c3cc69', 'PredictionString']
    if isinstance(MS_predictions, float):
        M_coords = []
    else:
        M_coords = str2coords(MS_predictions)
        
    new_M_coords = M_coords.copy()
    paired_M_coords = []
    for A_coord in A_coords:
        #print('')
        candidate = {'candidate':None, 'TD':99999999}
        for j in range(len(M_coords)):
            M_coord = M_coords[j]
            enough, TD, RD = is_close_enough(A_coord, M_coord)
            #print(enough)
            if enough and M_coord not in paired_M_coords and candidate['TD']>TD:
                candidate['candidate'] = M_coord
                candidate['TD'] = TD
                candidate['index'] = j
        if candidate['candidate'] == None:
            new_M_coords.append(A_coord)
        else:
            M_coord = candidate['candidate']
            paired_M_coords.append(M_coord)
            if merging_plan == 'weighted-add':
                new_coord = {}
                for item in ['pitch','yaw','roll','x','y','z']:
                    total_score = A_coord['confidence'] + M_coord['confidence']
                    new_coord[item] = (A_coord[item]*(A_coord['confidence']/total_score))\
                                    + (M_coord[item]*(M_coord['confidence']/total_score))       
            else:
                if A_coord['confidence'] > M_coord['confidence']:
                    new_coord = A_coord
                else:
                    new_coord = M_coord
            new_coord['confidence'] = 1 - ((1-A_coord['confidence'])*(1-M_coord['confidence']))
            new_M_coords[candidate['index']] = new_coord
    new_submission.loc[new_submission['ImageId']==auxiliary_submission.loc[i,'ImageId'], 'PredictionString'] = coords2str(new_M_coords)
    
new_submission.to_csv('./new_submission.csv', index=False)

# let's check the output

In [ ]:
for idx in range(20):
    
    img = imread(test_images_dir+new_submission['ImageId'].iloc[idx]+'.jpg')
    string1 = submission1.loc[submission1['ImageId']==new_submission['ImageId'].iloc[idx], 'PredictionString'].values[0]
    if isinstance(string1, float):
        string1 = ''
    string2 = submission2.loc[submission2['ImageId']==new_submission['ImageId'].iloc[idx], 'PredictionString'].values[0]
    if isinstance(string2, float):
        string2 = ''
    new_string = new_submission['PredictionString'].iloc[idx]
    if isinstance(new_string, float):
        new_string = ''
        
    coords1 = str2coords(string1)
    coords2 = str2coords(string2)
    new_coords = str2coords(new_string)
    
    fig, axes = plt.subplots(1, 3, figsize=(30,30))
    axes[0].set_title('Prediction 1')
    axes[0].imshow(visualize(img, coords1))
    axes[1].set_title('Prediction 2')
    axes[1].imshow(visualize(img, coords2))
    axes[2].set_title('New Prediction')
    axes[2].imshow(visualize(img, new_coords))
    plt.show()